In [1]:
%cd ..
%load_ext autoreload
%autoreload 2
%matplotlib notebook
from matplotlib import pyplot as plt
from rx import operators

from configs.config import GlobalConfig
GlobalConfig.initialize_global_configuration()

C:\Users\wuyua\PycharmProjects\RxExperiments


In [2]:
import logging
from utils.logging import configure_logger_to_output
logging_output = configure_logger_to_output(level=logging.INFO)
logging_output

Output()

In [3]:
from utils.mqtt_wrapper import MQTTClientWrapper

client = MQTTClientWrapper("raman_test", clean_session=False)
client.connect("workstation.local")
client.loop_start()

In [4]:
from source.raman_source import RamanSource

source = RamanSource(client).pipe(operators.share())

In [5]:
from sink.visualization_sink import PlotlyVisualizationSink
from sink.save_data_sink import SaveDataSink
from plotly import graph_objects as go
import pandas as pd
import datetime

visualization_sink = PlotlyVisualizationSink(name="Raman")
save_sink = SaveDataSink("raman", "raman")

source.pipe(
    operators.map(lambda x: [go.Scatter(x=x["wave_number"], y=x["count"], )])
).subscribe(visualization_sink)

source.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            [{
                "time": datetime.datetime.now(), 
                **dict(zip(x["wave_number"], x["count"]))
            }]
        )
    ),
).subscribe(save_sink)

visualization_sink.figure.update_xaxes(autorange="reversed")

display(visualization_sink.figure)

FigureWidget({
    'data': [], 'layout': {'template': '...', 'title': {'text': 'Raman'}, 'xaxis': {'autorange'…